In [90]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
%matplotlib inline
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [3]:
#load dataframes
# data: https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/overview
articles = pd.read_csv('data/articles.csv')
customers = pd.read_csv('data/customers.csv')
transactions = pd.read_csv('data/transactions_train.csv')

In [4]:
baskets = transactions.groupby(['customer_id', 't_dat'])['article_id'].apply(lambda x: ','.join(x.astype(str))).reset_index()

In [5]:
baskets

,customer_id,t_dat,article_id
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2018-12-27,"625548001,176209023,627759010"
1,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-05-02,697138006
2,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-05-25,"568601006,568601006"
3,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-07-25,"607642008,745232001"
4,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-09-18,656719005
...,...,...,...
9080174,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,2020-04-05,"821395003,806241002,714790020"
9080175,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,2020-04-09,"866755002,840360003"
9080176,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,2020-04-25,866755002
9080177,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,2020-06-22,882810001


In [6]:
baskets_list = []

for i in baskets["article_id"].values.tolist():
        baskets_list.append(i.split(","))

In [11]:
len(baskets_list)

9080179

In [27]:
new_list = []

for i in baskets_list:
    if len(i) > 1:
        new_list.append(i)

In [29]:
len(new_list)

6393136

In [98]:
articles.T

,0,1,2,3,4,5,6,7,8,9,...,105532,105533,105534,105535,105536,105537,105538,105539,105540,105541
article_id,108775015,108775044,108775051,110065001,110065002,110065011,111565001,111565003,111586001,111593001,...,949594001,950449002,952267001,952937003,952938001,953450001,953763001,956217002,957375001,959461001
product_code,108775,108775,108775,110065,110065,110065,111565,111565,111586,111593,...,949594,950449,952267,952937,952938,953450,953763,956217,957375,959461
prod_name,Strap top,Strap top,Strap top (1),OP T-shirt (Idro),OP T-shirt (Idro),OP T-shirt (Idro),20 den 1p Stockings,20 den 1p Stockings,Shape Up 30 den 1p Tights,Support 40 den 1p Tights,...,LOGG Elvis jogger.,Compact brush Fancy,Heavy plain overknee tights 1p,Jets dress,Elton top,5pk regular Placement1,SPORT Malaga tank,Cartwheel dress,CLAIRE HAIR CLAW,Lounge dress
product_type_no,253,253,253,306,306,306,304,302,273,304,...,272,78,304,265,254,302,253,265,72,265
product_type_name,Vest top,Vest top,Vest top,Bra,Bra,Bra,Underwear Tights,Socks,Leggings/Tights,Underwear Tights,...,Trousers,Other accessories,Underwear Tights,Dress,Top,Socks,Vest top,Dress,Hair clip,Dress
product_group_name,Garment Upper body,Garment Upper body,Garment Upper body,Underwear,Underwear,Underwear,Socks & Tights,Socks & Tights,Garment Lower body,Socks & Tights,...,Garment Lower body,Accessories,Socks & Tights,Garment Full body,Garment Upper body,Socks & Tights,Garment Upper body,Garment Full body,Accessories,Garment Full body
graphical_appearance_no,1010016,1010016,1010017,1010016,1010016,1010016,1010016,1010016,1010016,1010016,...,1010016,1010016,1010013,1010001,1010001,1010014,1010016,1010016,1010016,1010016
graphical_appearance_name,Solid,Solid,Stripe,Solid,Solid,Solid,Solid,Solid,Solid,Solid,...,Solid,Solid,Other pattern,All over pattern,All over pattern,Placement print,Solid,Solid,Solid,Solid
colour_group_code,9,10,11,9,10,12,9,13,9,9,...,8,50,9,13,13,9,9,9,9,11
colour_group_name,Black,White,Off White,Black,White,Light Beige,Black,Beige,Black,Black,...,Dark Grey,Other Pink,Black,Beige,Beige,Black,Black,Black,Black,Off White


In [102]:
len(articles['prod_name'])

105542

In [52]:
article_map = dict(zip(articles.article_id.values, articles.product_type_name.values))
article_map[625548001]

'Jacket'

In [54]:
new_list

[['625548001', '176209023', '627759010'],
 ['568601006', '568601006'],
 ['607642008', '745232001'],
 ['797065001', '797065001'],
 ['694736004', '785710001', '812683013'],
 ['841260003', '887593002', '890498002', '795440001', '859416011'],
 ['583558001', '639677008', '640244003'],
 ['521269001', '666448006'],
 ['583558001', '673677002'],
 ['723469005', '640174001', '700515001', '614854003'],
 ['724904001', '723529001', '640021012', '351484002'],
 ['658298001', '734282002'],
 ['640174001', '640021005', '724904002'],
 ['675319009', '631902006', '777690002'],
 ['778064010', '764280001', '478646001', '796137001'],
 ['723529001', '689898002', '351484002', '599580024'],
 ['559616014',
  '757974002',
  '615047003',
  '740943002',
  '689898002',
  '689898002',
  '758222003',
  '701561001',
  '745006005',
  '599580024',
  '699293009',
  '566140015'],
 ['611584007', '732842008', '640251009', '754356001'],
 ['866383001',
  '863103001',
  '590928022',
  '739590027',
  '739590027',
  '377277001',
  

In [55]:
purchase_list = []
for basket in new_list:
    basket_list = []
    for item in basket:
        #print(item)
        basket_list.append(article_map[int(item)])
    purchase_list.append(basket_list)

In [56]:
purchase_list

[['Jacket', 'Hoodie', 'Jacket'],
 ['Blazer', 'Blazer'],
 ['Shirt', 'Skirt'],
 ['Blazer', 'Blazer'],
 ['Gloves', 'Sweater', 'Dress'],
 ['T-shirt', 'Top', 'Jacket', 'Vest top', 'Sweater'],
 ['Dress', 'Shirt', 'Skirt'],
 ['Sweater', 'Sweater'],
 ['Dress', 'Sweater'],
 ['Bra', 'Sweater', 'Jacket', 'Underwear bottom'],
 ['Trousers', 'Bikini top', 'Trousers', 'Swimwear bottom'],
 ['Skirt', 'Skirt'],
 ['Sweater', 'Trousers', 'Trousers'],
 ['Blazer', 'Flat shoe', 'Necklace'],
 ['T-shirt', 'Vest top', 'Necklace', 'Vest top'],
 ['Bikini top', 'Bikini top', 'Swimwear bottom', 'Swimwear bottom'],
 ['Bikini top',
  'Top',
  'Top',
  'Dress',
  'Bikini top',
  'Bikini top',
  'Vest top',
  'Boots',
  'Sandals',
  'Swimwear bottom',
  'Bag',
  'Swimwear bottom'],
 ['Vest top', 'Trousers', 'Vest top', 'Bra'],
 ['Bikini top',
  'Cap/peaked',
  'Bikini top',
  'Swimwear bottom',
  'Swimwear bottom',
  'Vest top',
  'Shirt',
  'Dress',
  'Swimwear bottom'],
 ['Dress', 'Dress'],
 ['Cardigan',
  'Bikini to

In [68]:
#http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/
te = TransactionEncoder()
te_ary = te.fit(purchase_list).transform(purchase_list)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,Accessories set,Alice band,Baby Bib,Backpack,Bag,Ballerinas,Beanie,Belt,Bikini top,Blanket,...,Vest top,Wallet,Washing bag,Watch,Waterbottle,Wedge,Weekend/Gym bag,Wireless earphone case,Wood balls,Zipper head
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6393131,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6393132,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6393133,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6393134,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [71]:
frequent_tr_encoded = apriori(df, min_support=0.01, use_colnames=True)

In [106]:
frequent_itemsets = frequent_tr_encoded.sort_values(by=['support'], ascending=False)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
34,0.301383,(Trousers),1
10,0.222381,(Dress),1
29,0.220086,(Sweater),1
32,0.180339,(T-shirt),1
33,0.155071,(Top),1
...,...,...,...
12,0.010337,(Hair/alice band),1
144,0.010199,"(Top, Vest top, Trousers)",3
132,0.010190,"(Top, Dress, Sweater)",3
68,0.010056,"(Dress, Jumpsuit/Playsuit)",2


In [110]:
frequent_itemsets[ (frequent_itemsets['length'] == 2) ]

,support,itemsets,length
109,0.077951,"(Sweater, Trousers)",2
41,0.072424,"(Swimwear bottom, Bikini top)",2
77,0.060236,"(Dress, Trousers)",2
116,0.057402,"(T-shirt, Trousers)",2
119,0.053178,"(Top, Trousers)",2
...,...,...,...
105,0.010666,"(Sweater, Socks)",2
45,0.010595,"(Blazer, Sweater)",2
120,0.010553,"(Top, Underwear bottom)",2
42,0.010355,"(T-shirt, Bikini top)",2


In [111]:
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3).sort_values(by=['confidence'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(Swimwear bottom),(Bikini top),0.087251,0.087473,0.072424,0.830067,9.489455,0.064792,5.369924
2,(Bikini top),(Swimwear bottom),0.087473,0.087251,0.072424,0.827965,9.489455,0.064792,5.305606
28,"(Dress, Swimwear bottom)",(Bikini top),0.015921,0.087473,0.012747,0.800644,9.153093,0.011355,4.577387
29,"(Dress, Bikini top)",(Swimwear bottom),0.016228,0.087251,0.012747,0.785504,9.002796,0.011331,4.255311
7,(Underwear bottom),(Bra),0.090582,0.112359,0.041088,0.453596,4.037011,0.030910,1.624514
11,(Blazer),(Trousers),0.044468,0.301383,0.020088,0.451746,1.498909,0.006686,1.274258
41,(Jumpsuit/Playsuit),(Dress),0.022973,0.222381,0.010056,0.437743,1.968434,0.004947,1.383031
26,"(Blouse, Sweater)",(Trousers),0.031567,0.301383,0.013565,0.429727,1.425848,0.004051,1.225056
34,"(T-shirt, Blouse)",(Trousers),0.025220,0.301383,0.010690,0.423866,1.406401,0.003089,1.212594
27,"(Top, Blouse)",(Trousers),0.031759,0.301383,0.013087,0.412074,1.367277,0.003515,1.188274


In [112]:
oht_ary = te.fit(new_list).transform(new_list, sparse=True)
sparse_df = pd.DataFrame.sparse.from_spmatrix(oht_ary, columns=te.columns_)
sparse_df

,108775015,108775044,108775051,110065001,110065002,110065011,111565001,111565003,111586001,111593001,...,948152002,949198001,949551001,949551002,949594001,952267001,952938001,953450001,953763001,956217002
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6393131,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6393132,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6393133,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6393134,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [118]:
apriori(sparse_df, min_support=0.002, use_colnames=True, verbose=1)

Processing 462 combinations | Sampling itemset size 2


,support,itemsets
0,0.002524,(351484002)
1,0.004269,(372860001)
2,0.003294,(372860002)
3,0.002495,(399223001)
4,0.002328,(399256001)
5,0.002180,(448509014)
6,0.003080,(464297007)
7,0.002504,(562245001)
8,0.002481,(562245046)
9,0.002091,(568601006)
